# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [21]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import scann

In [22]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [23]:
config = pd.read_csv('config.csv').iloc[0]

In [24]:
config

Unnamed: 0                             0
id                                276633
k                                      5
dataset_file    ./Behance_subsampled.txt
output_file                 ./output.txt
Name: 0, dtype: object

In [25]:
user = config['id']
k_value = config['k']

### Read the Data

In [26]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [27]:
class IndexConverter:
    """
    Since, the user_id, picture_id column values are huge,
    we're replacing them with indices.
    """

    def __init__(self, df):
        self.user_id_val_to_index = {}
        self.picture_id_val_to_index = {}
        self.user_id_index_to_val = {}
        self.picture_id_index_to_val = {}
        for _, row in df.iterrows():
            if row['user_id'] not in self.user_id_val_to_index:
                index = len(self.user_id_val_to_index)
                self.user_id_val_to_index[row['user_id']] = index
                self.user_id_index_to_val[index] = row['user_id']
            if row['picture_id'] not in self.picture_id_val_to_index:
                index = len(self.picture_id_val_to_index)
                self.picture_id_val_to_index[row['picture_id']] = index
                self.picture_id_index_to_val[index] = row['picture_id']

    def apply(self, row):
        return np.array([self.user_id_val_to_index[row['user_id']],
                         self.picture_id_val_to_index[row['picture_id']]])

    def reverse_apply(self, row):
        return np.array([self.user_id_index_to_val['user_id'],
                         self.picture_id_index_to_val['picture_id']])

In [28]:
    df = pd.read_csv(config['dataset_file'], delim_whitespace=True,
                     header=None, names=['user_id', 'picture_id', 'timestamp'])
    df.drop('timestamp', axis=1, inplace=True)

    index_converter = IndexConverter(df)
    df_converted = df.apply(index_converter.apply,
                            axis=1, result_type='expand')
    df_converted_array = df_converted.to_numpy()

    sparse_matrix = np.zeros(
        (df_converted_array[:, 0].max() + 1, df_converted_array[:, 1].max() + 1), dtype=bool)
    sparse_matrix[df_converted_array[:, 0], df_converted_array[:, 1]] = 1

    columns = [f'picture_{i}' for i in range(sparse_matrix.shape[1])]
    spared_df = pd.DataFrame(sparse_matrix, columns=columns)
    print(spared_df)

     picture_0  picture_1  picture_2  picture_3  picture_4  picture_5  \
0         True       True       True      False      False      False   
1        False      False      False       True       True       True   
2        False      False      False      False      False      False   
3        False      False      False      False      False      False   
4        False      False      False      False      False      False   
..         ...        ...        ...        ...        ...        ...   
995      False      False      False      False      False      False   
996      False      False      False      False      False      False   
997      False      False      False      False      False      False   
998      False      False      False      False      False      False   
999      False      False      False      False      False      False   

     picture_6  picture_7  picture_8  picture_9  ...  picture_8292  \
0        False      False      False      False  ... 

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [29]:
user_likes = dict()

In [30]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

user_likes


{10356: [2126030, 629778, 2482777],
 15341: [54568,
  160555,
  201176,
  624115,
  1129883,
  2186353,
  2382516,
  2280270,
  2214882,
  2157975],
 18197: [2146091],
 19830: [1618145,
  1565965,
  1572127,
  1569221,
  1568687,
  1568621,
  1568803,
  1566159],
 28270: [2045757,
  1231321,
  2158487,
  2103664,
  528783,
  2167203,
  364321,
  667041,
  1960259,
  2167765,
  2230926,
  1768920,
  1630679,
  276451,
  823175,
  871122,
  1207647,
  863461,
  361889,
  356546,
  2253496,
  1240491,
  2292166,
  2301616,
  2297560,
  796951,
  2167139,
  635376,
  1155129,
  2378060,
  954142,
  1289701,
  802072,
  327267,
  2399532,
  94707,
  810562],
 32901: [2083554,
  462348,
  746753,
  2083654,
  574527,
  80939,
  178513,
  463636,
  357426,
  2152227,
  2236178,
  2248528,
  115744,
  2330202,
  2400898],
 33700: [1987715,
  1912973,
  1912751,
  899537,
  2114088,
  2167431,
  2288378,
  2369858,
  913618,
  2280270,
  1097443],
 34491: [1912751,
  2011825,
  244796,
  777551

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [31]:
normalized_Ds = spared_df/np.linalg.norm(spared_df,axis=1)[:,np.newaxis]
searcher = scann.scann_ops_pybind.builder(normalized_Ds,10,"dot_product").tree(num_leaves=200,num_leaves_to_search=100,training_sample_size=250000).score_ah(2,anisotropic_quantization_threshold=0.2).reorder(100).build()


In [32]:
outFile = open(config['output_file'], 'w')

for i in range(1000):
    neighbours,distances = searcher.search(spared_df.iloc[i],final_num_neighbors=k_value+1)
    picture_seti = set(user_likes[index_converter.user_id_index_to_val[i]])
    remaining_set = list()
    for j in range(1,6):
        a = index_converter.user_id_index_to_val[neighbours[j]]
        remaining_set.extend(user_likes[a])
        
    remaining_set = list(set(remaining_set)-picture_seti)
    for k in remaining_set:
        outFile.write(str(k) + ' ' + str(index_converter.user_id_index_to_val[i]) + '\n')

outFile.close()
        

### Answer the following questions:

#### Q1. **Explain how your choice of library works**

*your solution here*

#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*your solution here*

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*your solution here*